# 403 Iterate for CDR
In this notebook we will assess the new CDR needs based on an iterative search.

In [ ]:
ENSEMBLE_MEMBER = 4
MODEL='REMIND-MAgPIE 2.1-4.3'
SCENARIO='DeepElec_SSP2_ HighRE_Budg900'

In [ ]:
import json
import os
from pathlib import Path

import copy
import dotenv
import matplotlib.pyplot as plt
import openscm_runner
import scmdata
import pyam
import pandas as pd
import numpy as np
from scipy.optimize import minimize_scalar

import sys
sys.path.append('../scripts/')
from cdr import *

from climate_assessment.climate.wg3 import clean_wg3_scenarios
from climate_assessment.climate.magicc7 import get_magicc7_configurations
%load_ext autoreload
%autoreload 2

In [ ]:
dotenv.load_dotenv()

Step 1: Load the MAGICC configurations and select the ensemble member (climate realisation) that we want to run through for this study.

In [ ]:
os.environ['DYLD_LIBRARY_PATH'] = '/opt/homebrew/Cellar/gcc/13.2.0/lib/gcc/current/'
magicc_cfgs, _ = get_magicc7_configurations(
    magicc_version="v7.5.3",
    magicc_probabilistic_file=os.environ['MAGICC_AR6_PROBABILISTIC_DISTRIBUTION'],
    magicc_extra_config=None,
    num_cfgs=600,
    co2_and_non_co2_warming=False
)

In [ ]:
config_for_run = [magicc_cfgs[ENSEMBLE_MEMBER]]

Step 2: Load the necessary emissions-related files for this assessment. We need:
* The adapted novel CDR estimates lookup table
* The emission dataset to input to the climate model
* The original novel CDR estimates that we will use for the calibration iterations and the optimisation

In [ ]:
novel_cdr = pyam.IamDataFrame(
    Path(
        '../data/402_first_guess.csv'
    )
)

In [ ]:
emissions = pyam.IamDataFrame(
    Path(
        '../data/100_scenarios.csv'
    )
)

In [ ]:
metrics = pd.read_csv(
    Path(
        '../data/401_lookup.csv'
    ),
    index_col=[0,1, 2]
)

In [ ]:
novel_cdr_original = pyam.IamDataFrame(
    Path(
        '../data/100_novel_cdr.csv'
    )
)

Step 3: Filter both the datasets for the necessary model and scenario. Additionally, for the novel CDR dataframe, downselect it to get the ensemble member of choice.

In [ ]:
novel_cdr_for_iteration = (
    novel_cdr
    .filter(
        ensemble_member=ENSEMBLE_MEMBER,
        model=MODEL,
        scenario=SCENARIO
    )
)

In [ ]:
novel_cdr_for_iteration.interpolate(
    time=range(2015, 2101),
    inplace=True
)

In [ ]:
emissions_for_iteration = (
    emissions
    .filter(
        model=MODEL,
        scenario=SCENARIO
    )
)

In [ ]:
emissions_for_iteration.swap_time_for_year(inplace=True)

In [ ]:
novel_cdr_original_filtered = (
    novel_cdr_original
    .filter(
        model=MODEL,
        scenario=SCENARIO
    )
    .interpolate(
        time=range(2015, 2101)
    )
)

Step 4: We need the year of net zero CO2 and zero out the CDR before (so that we do not have a double counting effect)

In [ ]:
def process_novel_cdr(data, netzero_timing):
    data_to_return = (
        data.timeseries()
        -
        novel_cdr_original_filtered.timeseries()
    )
    return pyam.IamDataFrame(data_to_return)

In [ ]:
netzero_timing = metrics.loc[
    (MODEL, SCENARIO, ENSEMBLE_MEMBER),
    'netzero|CO2'
]

In [ ]:
novel_cdr_to_crunch = process_novel_cdr(
    novel_cdr_for_iteration,
    netzero_timing
)

In [ ]:
novel_cdr_for_iteration.set_meta(
    name='netzero|CO2',
    meta=netzero_timing
)

Step 4: Make a function that constructs a new emission dataframe to run through the simple climate model.

In [ ]:
def construct_new_emissions_dataframe(
        emissions, cdr
):
    emissions_to_return = copy.deepcopy(emissions)
    # Step 1: Get the CDR dataframe
    cdr_altered = pyam.IamDataFrame(
        cdr
        .timeseries()
        .reset_index()
        .drop(columns=cdr.extra_cols)
    )
    # Step 2: Pull out the emissions
    co2_ffi_var = 'AR6 climate diagnostics|Infilled|Emissions|CO2|Energy and Industrial Processes'
    co2_ffi = (
        emissions_to_return
        .filter(variable=co2_ffi_var)
    )
    # Step 3: Create a non CO2 dataframe
    nonco2_ffi = (
        emissions_to_return
        .filter(
            variable=co2_ffi_var,
            keep=False
        )
    )
    # Step 4: Aggregate the two variables
    concat = pyam.concat([cdr_altered, co2_ffi])
    concat_co2_ffi = concat.subtract(
        a=co2_ffi_var,
        b='Carbon Dioxide Removal|Novel',
        name=co2_ffi_var,
        ignore_units='Mt CO2/yr'
    )
    # Step 5: Return this
    df_to_return = pyam.concat([concat_co2_ffi, nonco2_ffi])
    return df_to_return.filter(year=range(2015, 2101))

In [ ]:
test = construct_new_emissions_dataframe(emissions_for_iteration, novel_cdr_to_crunch)

In [ ]:
fig, ax = plt.subplots()

test.filter(
    variable='*Emissions|CO2|Energy and Industrial Processes'
).plot(ax=ax, color='red')

emissions_for_iteration.filter(
    variable='*Emissions|CO2|Energy and Industrial Processes'
).plot(ax=ax, color='blue')

ax.set_title('')

Step 6: Create the input for MAGICC and run this through for a first cut.

In [ ]:
input_scm = scmdata.ScmRun(
    clean_wg3_scenarios(test)
)

In [ ]:
temp_confirmation =  openscm_runner.run(
    {'MAGICC7':config_for_run},
    input_scm,
    output_variables=[
        'Surface Temperature'
    ]
)

In [ ]:
temp_proc = (
    temp_confirmation
    .filter(region='World')
    .relative_to_ref_period_mean(
        year=[1850, 1900]
    )
    .timeseries()
)

In [ ]:
temp_calibrate_2100 = temp_proc['2100-01-01'].values[0].round(2)

In [ ]:
fig, ax = plt.subplots()

metrics_small = metrics.loc[
    (MODEL, SCENARIO, ENSEMBLE_MEMBER),
]

ax.bar(x=0, height=metrics_small['peak_warming'], width=0.1)
ax.bar(x=0.3, height=metrics_small['2100_warming'], width=0.1)
ax.bar(x=0.6, height=temp_calibrate_2100, width=0.1)
ax.axhline(1.5, color='black')

Step 7: Now that we know the direction (whether above or below), we now need a few calibration runs to get to the correct direction.

In [ ]:
deviation = temp_calibrate_2100 - 1.5
print(deviation)

In [ ]:
def return_temperature_deviation(input_emissions):
    # Step 1: Run this through the simple climate model
    temperatures = openscm_runner.run(
        {'MAGICC7':config_for_run},
        input_emissions,
        output_variables=[
            'Surface Temperature'
        ]    
    )
    # Step 2: Rebase to 1850-1900 and return the 2100 value deviation
    temperatures_rebased = (
        temperatures
        .filter(region='World')
        .relative_to_ref_period_mean(year=[1850, 1900])
    )
    temp_2100 = (
        temperatures_rebased
        .timeseries()['2100-01-01']
        .values[0]
    )
    # Step 3: Return the deviation
    return temp_2100 - 1.5

In [ ]:
novel_cdr_for_iteration.angle[0]
if deviation < 0:
    calibrate_type = 'reduce'
if deviation > 0:
    calibrate_type = 'increase'

In [ ]:
novel_cdr_for_iteration

In [ ]:
novel_cdr_original_filtered.meta = novel_cdr_for_iteration.meta

In [ ]:
def initial_calibration(
        emissions,
        cdr_first_guess,
        angle_first_guess,
        current_deviation
):
    # Step 1: Create a dictionary to store the calibrations
    calibration = {}
    emissions_copy = copy.deepcopy(emissions)
    cdr = copy.deepcopy(cdr_first_guess)
    # Step 2: Determine walk direction
    if current_deviation > 0:
        new_deviation = current_deviation
        step_size=0.5
        while new_deviation > 0:
            new_angle = angle_first_guess + step_size
            print(new_deviation, new_angle)
            if new_angle > 91:
                break
            new_cdr, _ = rotate_and_calc_cumulative(
                cdr,
                new_angle
            )
            new_emissions = (
                scmdata.ScmRun(
                    clean_wg3_scenarios(
                        construct_new_emissions_dataframe(
                            emissions_copy,
                            process_novel_cdr(new_cdr, cdr.meta['netzero|CO2'].values[0])
                        )
                    
                    )
                )
            )
            new_deviation = return_temperature_deviation(new_emissions)
            calibration[new_angle] = new_deviation
            step_size += step_size
    elif current_deviation < 0:
        step_size=5
        new_deviation = current_deviation
        while new_deviation < 0:
            new_angle = angle_first_guess - step_size
            if new_angle < 0:
                break
            print(new_deviation, new_angle)
            new_cdr, _ = rotate_and_calc_cumulative(
                cdr,
                new_angle
            )
            new_emissions = (
                scmdata.ScmRun(
                    clean_wg3_scenarios(
                        construct_new_emissions_dataframe(
                            emissions_copy,
                            process_novel_cdr(new_cdr, cdr.meta['netzero|CO2'].values[0])
                        )
                    
                    )
                )
            )
            new_deviation = return_temperature_deviation(new_emissions)
            calibration[new_angle] = new_deviation
            step_size += step_size
    return pd.DataFrame(
        calibration.items(),
        columns=['angle', 'deviation']
    )

In [ ]:
initial_calibration_set = initial_calibration(
    emissions_for_iteration,
    novel_cdr_original_filtered,
    novel_cdr_for_iteration.angle[0],
    current_deviation=deviation,
)

In [ ]:
initial_calibration_set

In [ ]:
def objective(angle, emissions, cdr_first_guess):
    # Step 1: Get the new CDR pathway
    new_cdr, _ = rotate_and_calc_cumulative(
        cdr_first_guess,
        angle
    )
    # Step 2: Get the new emissions pathway
    new_emissions = (
        scmdata.ScmRun(
            clean_wg3_scenarios(
                construct_new_emissions_dataframe(
                    emissions,
                    process_novel_cdr(new_cdr, cdr_first_guess.meta['netzero|CO2'].values[0])
                )        
            )
        )
    )
    new_deviation = abs(return_temperature_deviation(new_emissions))
    return new_deviation

In [ ]:
novel_cdr_for_iteration.angle[0]

In [ ]:
if calibrate_type == 'increase':
    try: max_angle = min(90, initial_calibration_set.iloc[-1]['angle'])
    except: max_angle=90
    bounds = (novel_cdr_for_iteration.angle[0], max_angle)
if calibrate_type == 'reduce':
    try: min_angle = max(0, initial_calibration_set.iloc[-1]['angle'])
    except: min_angle=0
    bounds = (min_angle, novel_cdr_for_iteration.angle[0])

In [ ]:
bounds

try:
    max_angle = min(90, initial_calibration_set.iloc[-1]['angle'])
except: max_angle=90

In [ ]:
result = minimize_scalar(
    objective,
    bounds=bounds,
    args=(emissions_for_iteration, novel_cdr_original_filtered),
    method='bounded'
)

In [ ]:
result

Last step: Make a function to construct and return the necessary CDR and temperature pathways.

In [ ]:
def objective(angle, emissions, cdr_first_guess):
    # Step 1: Get the new CDR pathway
    new_cdr, _ = rotate_and_calc_cumulative(
        cdr_first_guess,
        angle
    )
    # Step 2: Get the new emissions pathway
    new_emissions = (
        scmdata.ScmRun(
            clean_wg3_scenarios(
                construct_new_emissions_dataframe(
                    emissions,
                    process_novel_cdr(new_cdr, cdr_first_guess.meta['netzero|CO2'].values[0])
                )        
            )
        )
    )
    new_deviation = abs(return_temperature_deviation(new_emissions))
    return new_deviation

In [ ]:
def compile_necessary_output_files(result):
    # Step 1: Get the new CDR pathway
    new_cdr, cumulative_cdr = rotate_and_calc_cumulative(
        novel_cdr_original_filtered,
        result.x
    )
    new_cdr.set_meta(
        name='cumulative_cdr',
        meta=cumulative_cdr
    )
    # Step 2: Get the new emissions pathway
    new_emissions = (
        scmdata.ScmRun(
            clean_wg3_scenarios(
                construct_new_emissions_dataframe(
                    emissions_for_iteration,
                    process_novel_cdr(new_cdr, novel_cdr_original_filtered.meta['netzero|CO2'].values[0])
                )        
            )
        )
    )
    # Step 4: Calculate the warming
    new_warming = openscm_runner.run(
        {'MAGICC7':config_for_run},
        new_emissions,
        output_variables=[
            'Surface Temperature'
        ]    
    )
    # Step 4: Rebase and return
    new_warming_rebased = (
        new_warming
        .filter(region='World')
        .relative_to_ref_period_mean(year=[1850, 1900])
    )
    return new_cdr, new_warming_rebased

In [ ]:
cdr_pathway, temperature_pathway = compile_necessary_output_files(result)

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(10, 5))

# First, plot the CDR pathway
cdr_pathway.plot(ax=ax[0])
novel_cdr_original_filtered.plot(ax=ax[0], color='black', linestyle='--')

# Plot the temperature
temperature_pathway.filter(year=range(1850, 2100)).line_plot(ax=ax[1])
ax[1].axhline(1.5, color='black')

Save the files out for further assessment.

In [ ]:
cdr_pathway.to_csv(
    Path(
        f"results/CDR_{MODEL}_{SCENARIO}_{ENSEMBLE_MEMBER}.csv"
    )
)

In [ ]:
temperature_pathway.to_csv(
    Path(
        f"results/TEMP_{MODEL}_{SCENARIO}_{ENSEMBLE_MEMBER}.csv"
    )
)